In [1]:
!pip install kaggle

     ---------------------------------------- 0.0/82.7 kB ? eta -:--:--
     ---------------------------------------- 82.7/82.7 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for kaggle: filename=kaggle-1.6.17-py3-none-any.whl size=105802 sha256=d7bb043f17276214c6de794c78dfbdc44a8549077b968c17c79e5d2820155bd6
  Stored in directory: c:\users\hp\appdata\local\pip\cache\wheels\ff\55\fb\b27a466be754d2a06ffe0e37b248d844f090a63b51becea85d
Successfully built kaggle


In [1]:
import kaggle
import pandas as pd
import configparser
from sqlalchemy import create_engine
import psycopg2
from urllib.parse import quote_plus

In [4]:
!kaggle datasets download shivamb/netflix-shows -f netflix_titles.csv

Dataset URL: https://www.kaggle.com/datasets/shivamb/netflix-shows
License(s): CC0-1.0




  0%|          | 0.00/1.34M [00:00<?, ?B/s]
 75%|#######4  | 1.00M/1.34M [00:01<00:00, 672kB/s]
100%|##########| 1.34M/1.34M [00:01<00:00, 909kB/s]
100%|##########| 1.34M/1.34M [00:01<00:00, 841kB/s]


In [6]:
import zipfile
zip_ref = zipfile.ZipFile('netflix_titles.csv.zip')
zip_ref.extractall()
zip_ref.close()

In [2]:
df = pd.read_csv('data/netflix_titles.csv')

In [3]:
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [4]:
config = configparser.ConfigParser()

In [5]:
config.read('config.ini')

['config.ini']

In [6]:
dbname = config['database']['dbname']
user = config['database']['user']
password = config['database']['password']
port = int(config['database']['port'])
host = config['database']['host']

In [17]:
try:
    conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)
    print("Database connection successful with psycopg2")
except psycopg2.Error as e:
    print(f"Database connection error with psycopg2: {e}")

Database connection successful with psycopg2


In [18]:
cur = conn.cursor()

In [19]:
quoted_password = quote_plus(password)

In [20]:
engine = create_engine(f'postgresql+psycopg2://{user}:{quoted_password}@{host}:{port}/{dbname}')

In [51]:
try:
    cur.execute("""
    DROP TABLE IF EXISTS netflix_raw;
    CREATE TABLE netflix_raw(
    show_id varchar,
    type varchar,
    title varchar,
    director varchar,
    "cast" varchar,
    country varchar,
    date_added varchar,
    release_year varchar,
    rating int,
    duration varchar,
    listed_in varchar,
    description varchar
    );
    """)
    conn.commit()
    print('Table Created Successfully')
except psycopg2.Error as e:
    print(e)
finally:
    cur.close()
    conn.close()


Table Created Successfully


In [21]:
df.to_sql('netflix_extraction',con=engine,if_exists='replace',index=False)

807

In [23]:
df.isna().sum()

show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64